In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\Kidney_Disesae_Classification_with-MLFLOW_-_DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\Kidney_Disesae_Classification_with-MLFLOW_-_DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen= True)
class ModelTrainerConfig:
    root_dir: Path
    trained_model_path: Path
    base_model_updated: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_image_size: list
    params_augmentation: bool

In [7]:
from src.Kidney_Disease_Classification.constants import *
from src.Kidney_Disease_Classification.utils.common import read_yaml, create_directory

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directory([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        training = self.config.training
        params = self.params
        prepare_base_model = self.config.prepare_base_model
        training_data = os.path.join(self.config.data_ingestion.unzip_data_path, "kidney-ct-scan-image")

        create_directory([Path(training.root_dir)])

        model_trainer_config = ModelTrainerConfig(
            root_dir = Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            base_model_updated = Path(prepare_base_model.base_model_updated),
            training_data = training_data,
            params_epochs = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_image_size = params.IMAGE_SIZE,
            params_augmentation = params.AUGMENTATION
        )
        return model_trainer_config

In [9]:
import tensorflow as tf

In [10]:
class ModelTraining:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.base_model_updated
        )
    
        

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
            )
        
        dataflow_kwargs = dict(
            batch_size = self.config.params_batch_size,
            target_size =self.config.params_image_size[:-1],
            interpolation = "bilinear"
            )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs            
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
            )
        
        if self.config.params_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                           
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,                
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                **datagenerator_kwargs
                
                )
            
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset= "training",
            shuffle= False,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size


        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator
        )

        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )

In [13]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_training = ModelTraining(config= model_trainer_config)
    model_training.get_base_model()
    model_training.train_valid_generator()
    model_training.train()
except Exception as e:
    raise e 

[2023-11-15 18:37:28,485  - INFO - common - Reading done from config\config.yaml file]
[2023-11-15 18:37:28,497  - INFO - common - Reading done from params.yaml file]
[2023-11-15 18:37:28,501  - INFO - common - Directory is created for artifacts]
[2023-11-15 18:37:28,501  - INFO - common - Directory is created for artifacts\training]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
23/23 [==============================] - 84s 4s/step - loss: 25.5073 - accuracy: 0.5843 - val_loss: 9.1449 - val_accuracy: 0.6000
